<a href="https://colab.research.google.com/github/viniciusvmda/procedural-texture/blob/master/gp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programação Genética
https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/

## Instalação
<code>pip install deap
sudo apt-get install python-dev graphviz libgraphviz-dev pkg-config
pip install pygraphviz networkx
pip install noise
pip install imagen
pip install opencv-python --user</code>

## Bibliotecas

In [21]:
# Genetic Programming
from deap import base, creator, gp, tools, algorithms

# Graphics
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from matplotlib import pyplot as plt

# Computer vision
import cv2

# Util
import numpy as np
import random
import math
import operator   # math basic operations
from timeit import default_timer as timer
from copy import deepcopy
import warnings
import sys
# from numba import jit

# Operators
from noise import snoise2
import imagen as ig   # Pattern generation
from sklearn.cluster import KMeans   # get dominant colors

# Upload images
# from google.colab import files
# from io import BytesIO
# from google.colab.patches import cv2_imshow

## Desenvolvimento

### Constantes

In [30]:
# Image constants
NUMBER_OF_CHANNELS = 3
RGB_MAX = 255;

# Operators
MAX_OCTAVES = 5   # perlin noise
MIN_SIZE_OF_STRIPES = 30
N_DOMINANT_COLORS = 3
IMG_SIZE_TO_COMPARE_COLORS = 30
MARGIN_OF_ERROR = 20
MAX_WEIGHT_ON_IMG_ADD = 10   # addWeighted
MAX_PARAM_VALUE = 1000   # float operators
# Evaluation
CANNY_THRESHOLD_MIN_DETAILS = (170, 200)
CANNY_THRESHOLD_MAX_DETAILS = (10, 40)
NUMBER_OF_MATCHES = 60

# GP parameters
N_INITIAL_POPULATION = 150
N_GENERATIONS = 15
N_INDIVIDUALS_NEXT_GENERATION = N_INITIAL_POPULATION
N_CHILDREN_NEXT_GENERATION = N_INITIAL_POPULATION
N_HALL_OF_FAME_INDIVIDUALS = 1
CROSSOVER_PROBABILITY = 0.7
MUTATION_PROBABILITY = 0.1
MIN_TREE_SIZE_INIT_POPULATION = 2
MAX_TREE_SIZE_INIT_POPULATION = 6
MIN_TREE_SIZE_MUTATION = 2
MAX_TREE_SIZE_MUTATION = 4
TOURNAMENT_SIZE = 5
# Execution parameters
NUMBER_OF_TESTS_PER_TARGET = 2

random.seed(39)
tiles = ig.SquareGrating()

### Util

In [23]:
def plotTree(expr):
  nodes, edges, labels = gp.graph(expr)
  
  g = nx.DiGraph()
  g.add_nodes_from(nodes)
  g.add_edges_from(edges)
  pos = graphviz_layout(g, prog="dot")

  nx.draw_networkx_nodes(g, pos)
  nx.draw_networkx_edges(g, pos)
  nx.draw_networkx_labels(g, pos, labels)
  
  plt.show()


def convertFloatToUint8(img):
  return cv2.normalize(img, None, RGB_MAX, 0, cv2.NORM_MINMAX, cv2.CV_8UC3);


def createHistogram(cluster):
  numLabels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
  hist, _ = np.histogram(cluster.labels_, bins=numLabels)
  hist = hist.astype('float32')
  hist /= hist.sum()
  return hist
  

def getDominantColorsKmeans(img):
  imgResized = cv2.resize(img, (IMG_SIZE_TO_COMPARE_COLORS, IMG_SIZE_TO_COMPARE_COLORS))
  imgReshaped = imgResized.reshape((imgResized.shape[0] * imgResized.shape[1], imgResized.shape[2]))

  clusters = KMeans(n_clusters=N_DOMINANT_COLORS).fit(imgReshaped)
  # count the dominant colors and put them in "buckets"
  histogram = createHistogram(clusters)

  # Slice cluster array to the size of histogram
  clusterCentersSliced = clusters.cluster_centers_[:histogram.shape[0]]
  clusterIndexes = np.arange(clusterCentersSliced.shape[0])

  # then sort them, most-common first
  combined = np.column_stack((histogram, clusterIndexes))
  combined = sorted(combined, key=operator.itemgetter(0), reverse=True)

  dominantColors = []
  for [_, colorFloatIndex] in combined:
    color = clusterCentersSliced[int(colorFloatIndex)].astype(np.uint8)
    dominantColors.append(color)

  return dominantColors


def getColourHistogramMatching(img1, img2):
  h_bins = 50
  s_bins = 60
  histSize = [h_bins, s_bins]
  # hue varies from 0 to 179, saturation from 0 to 255
  h_ranges = [0, 180]
  s_ranges = [0, 256]
  ranges = h_ranges + s_ranges # concat lists
  # Use the 0-th and 1-st channels
  channels = [0, 1]

  img1Hsv = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
  img2Hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

  img1Hist = cv2.calcHist([img1Hsv], channels, None, histSize, ranges, accumulate=False)
  cv2.normalize(img1Hist, img1Hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
  img2Hist = cv2.calcHist([img2Hsv], channels, None, histSize, ranges, accumulate=False)
  cv2.normalize(img2Hist, img2Hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
  
  similarity = np.abs(cv2.compareHist(img1Hist, img2Hist, cv2.HISTCMP_CORREL))
  return similarity


def getOrbMatch(img, targetKp, targetDes):
  kp, des = getKeypoinstAndDescriptors(img)
  if targetDes is None or des is None:
    return 0
    
  numberOfMatches = min(len(targetDes), len(des), NUMBER_OF_MATCHES) 

  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
  matches = bf.match(targetDes[:numberOfMatches], des[:numberOfMatches])
  matches = sorted(matches, key = lambda x:x.distance)
  
  maxValue = math.sqrt(img.shape[0] * img.shape[1]) * numberOfMatches
  total = 0
  
  for match in matches:
    total += match.distance
  
  return 1 - (total/maxValue)


def getKeypoinstAndDescriptors(img):
  for cannyRange in [CANNY_THRESHOLD_MIN_DETAILS, CANNY_THRESHOLD_MAX_DETAILS]:
    edgesMin = cv2.Canny(img, cannyRange[0], cannyRange[1])
    kp, des = runOrb(edgesMin)
    if des is not None:
      return kp, des
    
  return None, None
  

def runOrb(img):
  orb = cv2.ORB_create()
  # find the keypoints with ORB
  kp = orb.detect(img, None)
  # compute the descriptors with ORB
  kp, des = orb.compute(img, kp)
    
  return kp, des

### Operadores

#### Imagem

In [25]:
class ImageOperator:
  
  def __init__(self, targetImg):
    self.targetImg = targetImg
    self.targetImgWidth = targetImg.shape[0];
    self.targetImgHeight = targetImg.shape[1];
    self.targetDominantColors = getDominantColorsKmeans(targetImg)

  
  def perlinNoise(self, scale, octaves):
    positiveScale = scale + 1.0
    octavesNormalized = int(octaves) % MAX_OCTAVES + 1

    output = np.zeros((self.targetImgWidth, self.targetImgHeight), dtype=np.float)
    for x in range(0, self.targetImgWidth):
      for y in range(0, self.targetImgHeight):
        noiseValue = snoise2(x/positiveScale, y/positiveScale, octaves=octavesNormalized, base=0)
        output[x][y] = noiseValue
    outputUint8 = convertFloatToUint8(output)
    return cv2.cvtColor(outputUint8, cv2.COLOR_GRAY2BGR)


  def createStripes(self, img, numberOfTilesFloat, orientation):
    newImg = deepcopy(img)
    maxNumberOfLines = int(self.targetImgWidth / MIN_SIZE_OF_STRIPES)
    numberOfTiles = int(numberOfTilesFloat) % maxNumberOfLines + 1

    lines = tiles(xdensity=self.targetImgWidth, ydensity=self.targetImgHeight, phase=np.pi/2, frequency=numberOfTiles, orientation=orientation)
    linesUint8 = convertFloatToUint8(lines)
    
    for x in range(0, self.targetImgWidth):
      for y in range(0, self.targetImgHeight):
        if linesUint8[x][y] < MARGIN_OF_ERROR:
          newImg[x][y] = np.full(NUMBER_OF_CHANNELS, linesUint8[x][y])

    return newImg

  
  def createChessBoard(self, img, blockSizeFloat):
    newImg = deepcopy(img)
    blockSize = int(blockSizeFloat % (self.targetImgWidth - 1)) + 1

    xBegin = int((blockSize - self.targetImgWidth % blockSize) / 2)
    yBegin = int((blockSize - self.targetImgHeight % blockSize) / 2)  
    xBegin = xBegin if ((self.targetImgWidth - xBegin) / blockSize) % 2 == 0 else -xBegin
    yBegin = yBegin if ((self.targetImgHeight - yBegin) / blockSize) % 2 == 0 else -yBegin

    for column in range(2):  
      for i in range(xBegin + column * blockSize, self.targetImgWidth, 2 * blockSize):
        for j in range(yBegin + column * blockSize, self.targetImgHeight, 2 * blockSize):
          x = i if i >= 0 else 0
          y = j if j >= 0 else 0
          newImg[y:j + blockSize, x:i + blockSize] = 0

    return newImg


  # Remove noise while preserving edges
  def bilateralFilter(self, img, sigmaValues):
    sigmaMinValue = 10   # there are no changes with values lower than 10
    filterSize = 5   # Recomended value for diameter of each pixel neighborhood used during filtering
    sigmaValuesPositive = sigmaValues + sigmaMinValue   # define how pixels will be mixed
    return cv2.bilateralFilter(img, filterSize, sigmaValuesPositive, sigmaValuesPositive)


  # Erode the boundaries of the objects in the image 
  def erodeImage(self, img, kernelSizeFloat):
    kernel = self.getKernelFromFloatValue(kernelSizeFloat)
    numberOfExecutions = 1
    return cv2.erode(img, kernel, iterations = numberOfExecutions)


  # Dilate the boundaries of the objects in the image 
  def dilateImage(self, img, kernelSizeFloat):
    kernel = self.getKernelFromFloatValue(kernelSizeFloat)
    numberOfExecutions = 1
    return cv2.dilate(img, kernel, iterations = numberOfExecutions)


  def getKernelFromFloatValue(self, kernelSizeFloat):
    maxKernelValue = int(self.targetImgWidth * 0.05)
    kernelSize = int(kernelSizeFloat) % maxKernelValue + 1
    return np.ones((kernelSize, kernelSize), np.uint8)   # window that slides through the image
  
  
  def colorizeImage(self, img):
    newImg = deepcopy(img)
    currentDominantColors = getDominantColorsKmeans(img)

    for x in range(newImg.shape[0]):
      for y in range(newImg.shape[1]):
        for i in range(len(currentDominantColors)):
            isInside = self.isColorInsideMargin(img[x][y], currentDominantColors[i])
            if (isInside):
              newImg[x][y] = self.targetDominantColors[i]
              i += 1
              
    return newImg


  def isColorInsideMargin(self, color, currentDominantColor):
    currentDominantColorFloat = currentDominantColor.astype(np.float)
    lowestValue = currentDominantColorFloat - MARGIN_OF_ERROR
    greatesValue = currentDominantColorFloat + MARGIN_OF_ERROR
    comparison = np.logical_and(color >= lowestValue, color <= greatesValue)
    return np.all(comparison)
  
  
  def addWeighted(self, img1, img2, weightFloat):
    weight = (weightFloat % MAX_WEIGHT_ON_IMG_ADD + 1) / MAX_WEIGHT_ON_IMG_ADD  
    scalarToSum = 0
    return cv2.addWeighted(img1, weight, img2, 1.0 - weight, scalarToSum)
  
  
  def applyCanny(self, img):
    edges = cv2.Canny(img, CANNY_THRESHOLD_MIN_DETAILS[0], CANNY_THRESHOLD_MIN_DETAILS[1])
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

#### Float

In [26]:
class FloatOperator:

  def protectedAdd(left, right):
      return (left + right) % MAX_PARAM_VALUE
  
  
  def protectedSub(left, right):
      return abs((left - right) % MAX_PARAM_VALUE )
  

  def protectedMul(left, right):
    return (left * right) % MAX_PARAM_VALUE

  
  def protectedDiv(left, right):
      return (left / right if right != 0.0 else left)


  def protectedMod(left, right):
      return (left % right if right != 0.0 else left)


  def protectedLog(num, base):
    try:
        return abs(math.log(num, base))
    except:
        return 1      

  def protectedSin(num):
    return abs(math.sin(num))
  
  
  def protectedCos(num):
    return abs(math.cos(num))
  
  
  def avg(num1, num2):
    return (num1 + num2) / 2.0

### Gerador de Texturas

In [27]:
class TextureGenerator:

  def __init__(
      self, targetImg, nInitialPopulation, nGenerations, nIndividualsNextGeneration, nChildrenNextGeneration,
      nHallOfFameIndividuals, crossoverProbability, mutationProbability
  ):
    self.targetImg = targetImg
    self.nInitialPopulation = nInitialPopulation
    self.nGenerations = nGenerations
    self.nIndividualsNextGeneration = nIndividualsNextGeneration
    self.nChildrenNextGeneration = nChildrenNextGeneration
    self.nHallOfFameIndividuals = nHallOfFameIndividuals
    self.crossoverProbability = crossoverProbability
    self.mutationProbability = mutationProbability
    # Define input types and output type
    self.pset = gp.PrimitiveSetTyped("main", [], np.ndarray)
    # Init toolbox
    self.toolbox = base.Toolbox()
    # Compute values
    self.targetKp, self.targetDes = getKeypoinstAndDescriptors(targetImg)

  
  def setImageOperators(self):
    imageOperator = ImageOperator(self.targetImg)
    self.pset.addPrimitive(imageOperator.perlinNoise, [float, float], np.ndarray, "ruído")
    self.pset.addPrimitive(imageOperator.createStripes, [np.ndarray, float, float], np.ndarray, "listras")
    self.pset.addPrimitive(imageOperator.createChessBoard, [np.ndarray, float], np.ndarray, "xadrez")
    self.pset.addPrimitive(imageOperator.colorizeImage, [np.ndarray], np.ndarray, "rgb")
    self.pset.addPrimitive(imageOperator.addWeighted, [np.ndarray, np.ndarray, float], np.ndarray, "somaImg")
    self.pset.addPrimitive(imageOperator.bilateralFilter, [np.ndarray, float], np.ndarray, "bilateral")
    self.pset.addPrimitive(imageOperator.erodeImage, [np.ndarray, float], np.ndarray, "erodir")
    self.pset.addPrimitive(imageOperator.dilateImage, [np.ndarray, float], np.ndarray, "dilatar")
#     self.pset.addPrimitive(imageOperator.applyCanny, [np.ndarray], np.ndarray, "canny")


  def setFloatOperators(self):
    self.pset.addPrimitive(FloatOperator.protectedAdd, [float, float], float, "soma")
    self.pset.addPrimitive(FloatOperator.protectedSub, [float, float], float, "sub")
    self.pset.addPrimitive(FloatOperator.protectedMul, [float, float], float, "mult")
    self.pset.addPrimitive(FloatOperator.protectedDiv, [float, float], float, "div")
    self.pset.addPrimitive(FloatOperator.protectedMod, [float, float], float, "mod")
    self.pset.addPrimitive(FloatOperator.protectedLog, [float, float], float, "log")
    self.pset.addPrimitive(FloatOperator.protectedSin, [float], float, "sen")
    self.pset.addPrimitive(FloatOperator.protectedCos, [float], float, "cos")
    self.pset.addPrimitive(FloatOperator.avg, [float, float], float, "avg")
    self.pset.addPrimitive(min, [float, float], float, "min")
    self.pset.addPrimitive(max, [float, float], float, "max")

    
  def setTerminals(self):
    WHITE_IMG = np.full(self.targetImg.shape, RGB_MAX, dtype=np.uint8)
    self.pset.addTerminal(WHITE_IMG, np.ndarray, "WHITE")  # required terminal
    
    previous = 1
    num = 1
    while num < 100:
      self.pset.addTerminal(float(num), float, str(num))  # required terminal
      aux = num
      num += previous
      previous = aux

  
  def evalFitness(self, individual):
    # Transform the tree expression in a callable function
    texture = self.toolbox.compile(expr=individual)
    
    fitness1 = getOrbMatch(texture, self.targetKp, self.targetDes)
    fitness2 = getColourHistogramMatching(self.targetImg, texture)
    
    return (0.85 * fitness1 + 0.15 * fitness2),


  def setGenneticProgrammingOperators(self):
    # Define fitness with one objective
    creator.create("FitnessMultiMax", base.Fitness, weights=(1.0,))
    # Create individual and add primitiveSet and fitness
    creator.create("Individual", gp.PrimitiveTree, pset=self.pset, fitness=creator.FitnessMultiMax)
    
    # Pupulation
    self.toolbox.register("expr", gp.genHalfAndHalf, pset=self.pset, min_=MIN_TREE_SIZE_INIT_POPULATION, max_=MAX_TREE_SIZE_INIT_POPULATION)
    self.toolbox.register("individual", tools.initIterate, creator.Individual, self.toolbox.expr)
    self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
    # Fitness Evaulation
    self.toolbox.register("evaluate", self.evalFitness)
    # Selection
    self.toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)
#     Crossover
#     self.toolbox.register("mate", gp.cxOnePointLeafBiased, termpb=0.1)
    self.toolbox.register("mate", gp.cxOnePoint)
    
    # Mutation
    self.toolbox.register("expr_mut", gp.genHalfAndHalf, min_=MIN_TREE_SIZE_MUTATION, max_=MAX_TREE_SIZE_MUTATION)
    self.toolbox.register("mutate", gp.mutUniform, expr=self.toolbox.expr_mut, pset=self.pset)
    # Compile function
    self.toolbox.register("compile", gp.compile, pset=self.pset)


  def initGenneticProgramming(self):
    self.setImageOperators()
    self.setFloatOperators()
    self.setTerminals()
    self.setGenneticProgrammingOperators()
    
    
  def getElapsedTime(self, _):
    elapsedTime = timer() - self.generationStartTime
    self.generationStartTime = timer()
    return elapsedTime
    
  def runGenneticProgramming(self):
    self.initGenneticProgramming()

    population = self.toolbox.population(n=self.nInitialPopulation)
    self.hallOfFame = tools.HallOfFame(self.nHallOfFameIndividuals)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("std", np.std)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
        
    stats.register("time", self.getElapsedTime)
    
    start = timer()
    self.generationStartTime = start
    result = algorithms.eaMuPlusLambda(
        population, self.toolbox, self.nIndividualsNextGeneration, self.nChildrenNextGeneration, 
        self.crossoverProbability, self.mutationProbability, self.nGenerations, stats, self.hallOfFame
    )
#     result = algorithms.eaSimple(
#         population, self.toolbox, self.crossoverProbability, self.mutationProbability, self.nGenerations, stats, self.hallOfFame
#     )
    executionTime = timer() - start
    print("Elapsed time: " + str(executionTime) + " seconds")
    return result
    
  
  def generateTexture(self, nIndividual):
    tree = gp.PrimitiveTree(self.hallOfFame[nIndividual])
    generatedTexture = gp.compile(tree, self.pset)
    return generatedTexture
  
  
  def getResultantExpression(self, nIndividual):
    return self.hallOfFame[nIndividual]
      

In [28]:
def writeOutputOnFile(logbook, expression, imageName, testNumber):
  generations = logbook.select("gen")
  generationDuration = logbook.select("time")
  
  fitnessMin = logbook.select("min")
  fitnessMax = logbook.select("max")
  fitnessAvg = logbook.select("avg")
  fitnessStd = logbook.select("std")

  file = open('output/' + imageName + str(testNumber) + '.csv',"w+")
  file.write(str(expression) + "\n\n")

  for i in range(len(generations)):
    file.write("%d" % (generations[i]))
    file.write(',' + '{0:.5f}'.format(fitnessMin[i]))
#     file.write(',' + '{0:.5f}'.format(fitnessMin[i][1]))
    file.write(',' + '{0:.5f}'.format(fitnessMax[i]))
#     file.write(',' + '{0:.5f}'.format(fitnessMax[i][1]))
    file.write(',' + '{0:.5f}'.format(fitnessAvg[i]))
#     file.write(',' + '{0:.5f}'.format(fitnessAvg[i][1]))
    file.write(',' + '{0:.5f}'.format(fitnessStd[i]))
#     file.write(',' + '{0:.5f}'.format(fitnessStd[i][1]))
    file.write(',' + '{0:.2f}'.format(generationDuration[i]) + '\n')

  file.close()


def writeTextureOnFile(generateTexture, individualNumber, imageName, testNumber):
  texture = generateTexture(individualNumber)
  fileName = 'output/' + imageName + str(testNumber) + '.jpg'
  cv2.imwrite(fileName, texture)

### Main

In [ ]:
images = ['sky', 'chess', 'water', 'brick', 'lava', 'grass', 'granite', 'wood']

print('Process started')
with warnings.catch_warnings():
  warnings.simplefilter("ignore")
  
  for imageName in images:
    targetImg = cv2.imread('images/' + imageName + '.jpg')

    for testNumber in range(NUMBER_OF_TESTS_PER_TARGET):
      textureGenerator = TextureGenerator(
          targetImg, N_INITIAL_POPULATION, N_GENERATIONS, N_INDIVIDUALS_NEXT_GENERATION, 
          N_CHILDREN_NEXT_GENERATION, N_HALL_OF_FAME_INDIVIDUALS, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY
      )
      
      result = textureGenerator.runGenneticProgramming()
      logbook = result[1]
      individualNumber = 0
      
      expression = textureGenerator.getResultantExpression(individualNumber)
      writeOutputOnFile(logbook, expression, imageName, testNumber)
      writeTextureOnFile(textureGenerator.generateTexture, individualNumber, imageName, testNumber)

print('Process finished')

Process started
gen	nevals	std     	avg     	min        	max     	time   
0  	150   	0.245238	0.379052	0.000256135	0.617308	120.964
1  	119   	0.0252351	0.568289	0.460285   	0.617308	91.7567
2  	125   	0.0193434	0.596597	0.547001   	0.649624	158.52 
3  	112   	0.0187613	0.616735	0.587896   	0.687998	222.897
4  	111   	0.0229478	0.639471	0.602025   	0.687998	312.073
5  	121   	0.0169321	0.668419	0.617308   	0.688202	402.126
6  	115   	0.0084801	0.684588	0.649624   	0.699376	480.89 
7  	126   	0.00445746	0.690551	0.684686   	0.703785	648.847
8  	120   	0.00547775	0.694995	0.687998   	0.703857	681.143
9  	118   	0.00358832	0.70082 	0.691331   	0.706848	765.829
10 	123   	0.00227361	0.703622	0.692021   	0.712339	968.11 
11 	121   	0.00291798	0.706074	0.702349   	0.716714	994.23 
12 	122   	0.00634072	0.70915 	0.703785   	0.75392 	1059.02
13 	125   	0.0120742 	0.715467	0.689266   	0.75392 	981.988
14 	117   	0.0148959 	0.722387	0.706159   	0.75392 	943.42 
15 	131   	0.0178832 	0.735159	0.7

In [32]:
img = cv2.imread('images/brick.jpg')
imgOp = ImageOperator(img)

img2 = imgOp.createStripes(np.full((256,256,3), 255, dtype=np.uint8), 100, math.pi/2)
img3 = imgOp.colorizeImage(img2)
cv2.imwrite('brick.jpg', img)
cv2.imwrite('stripes.jpg', img2)
cv2.imwrite('exRgb.jpg', img3)

True